In [1]:
import os
import pandas as pd
import re
import meshio
import tqdm

In [2]:
df_pos = pd.read_csv('SciVisContest23/viz-no-network/positions/rank_0_positions.txt', sep=' ', names=['local_id', 'x', 'y', 'z', 'area', 'type'], skiprows=8)

In [3]:
points = df_pos[['x', 'y', 'z']].values
cells = {}

In [5]:
columns = ['step', 'fired', 'fired_fraction', 'activity', 'dampening', 'current_calcium', 'target_calcium', 'synaptic_input', 'background_input', 
    'grown_axons', 'connected_axon', 'grown_dendrites', 'connected_dendrites', 'id']
df = pd.DataFrame(columns=['id', 'step', 'fired', 'activity', 'current_calcium', 'synaptic_input', 'connected_dendrites'])
files = os.listdir('SciVisContest23/viz-no-network/monitors')
list_df = []
for file in tqdm.tqdm(files):
    df_inter = pd.read_csv('SciVisContest23/viz-no-network/monitors/' + file, sep=';', names=columns)
    df_inter['id'] = re.search(r'\d+_(\d+)\.csv', file).group(1)
    df_inter = df_inter[['id', 'step', 'fired', 'activity', 'current_calcium', 'synaptic_input', 'connected_dendrites']]
    list_df.append(df_inter)
df = pd.concat(list_df, axis=0)

df = df.astype({
    'id': int,
    'step': int, 
    'fired': int,
    'activity': float,
    'current_calcium': float,
    'synaptic_input': float,
    'connected_dendrites': int
})

df.to_csv('SciVisContest23/viz-no-network/timeseries/neurons.csv')

100%|██████████| 50000/50000 [30:08<00:00, 27.64it/s]  


In [6]:
df = pd.read_csv('SciVisContest23/viz-no-network/timeseries/neurons.csv')

In [7]:
ids = set(range(0, 50000))
for step in tqdm.tqdm(df['step'].unique()):
    df_inter = df[df['step'] == step][['id', 'step', 'fired', 'activity', 'current_calcium', 'synaptic_input', 'connected_dendrites']]
    ids_mis = ids - set(df_inter['id'].unique())
    df_mis = pd.DataFrame({'id': list(ids_mis), 'fired': 0})
    df_inter = df_inter.astype({
        'id': int,
        'fired': int,
        'activity': float,
        'current_calcium': float,
        'synaptic_input': float,
        'connected_dendrites': int
    })
    df_uni = pd.concat([df_inter, df_mis], ignore_index=True)
    df_uni = df_uni.sort_values('id')
    point_data = {
        'fired': df_uni['fired'].values,
        'activity': df_uni['activity'].values,
        'current_calcium': df_uni['current_calcium'].values,
        'synaptic_input': df_uni['synaptic_input'].values,
        'connected_dendrites': df_uni['connected_dendrites'].values
    }
    mesh = meshio.Mesh(points=points, cells=cells, point_data=point_data)
    mesh.write('SciVisContest23/viz-no-network/timeseries/neurons/step_full_' + str(f"{int(step):06d}") + '.vtk')

100%|██████████| 10000/10000 [2:26:07<00:00,  1.14it/s] 


: 